This script reproduces what the main.py function does but divided into its parts so we can visualize the whole process

In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
import argparse
import pickle
from ucimlrepo import fetch_ucirepo 

In [2]:
from src.dataset import Data_handling
from src.weakener import Weakener
from src.model import MLP
from utils.datasets_generation import generate_dataset
import utils.losses as losses
from utils.train_test_loop import train_and_evaluate

In [3]:
reps = 10
dataset_base_path = 'Datasets/weak_datasets'
dataset = 'image'
corruption = 'pll'
corr_p = 0.5
corr_n = None

#for i in range(reps):
#        generate_dataset(dataset=dataset,corruption=corruption,corr_p=corr_p,repetitions=i)


In [4]:

reps =10
dataset_base_path = 'Datasets/weak_datasets'
dataset = 'image'
corruption = 'pll'
corr_p = 0.5
corr_n = None
loss_type = 'Forward'
for i in range(reps):
    base_dir = "Datasets/weak_datasets"
    if corr_n is not None:
        folder_path = os.path.join(base_dir, f'{dataset}_{corruption}_p_+{corr_p}p_-{corr_n}')
    else:
        folder_path = os.path.join(base_dir, f'{dataset}_{corruption}_p{corr_p}')
    f = open(folder_path + f'/Dataset_{i}.pkl','rb')
    Data,Weak = pickle.load(f)
    f.close()

    
    
    if loss_type == 'Backward':
        loss_fn = losses.FwdBwdLoss(Weak.Y,np.eye(Weak.c))
    elif loss_type == 'Forward':
        loss_fn = losses.FwdBwdLoss(np.eye(Weak.d),Weak.M)
    elif loss_type == 'EM':
        loss_fn = losses.EMLoss(Weak.M)
    elif loss_type == 'LBL':
        loss_fn = losses.LBLoss()
    elif loss_type == 'Backward_opt':
        loss_fn = losses.FwdBwdLoss(Weak.Y_opt,np.eye(Weak.c))
    elif loss_type == 'Backward_conv':
        loss_fn = losses.FwdBwdLoss(Weak.Y_conv,np.eye(Weak.c))
    elif loss_type == 'Backward_opt_conv':
        loss_fn = losses.FwdBwdLoss(Weak.Y_opt_conv,np.eye(Weak.c))
    elif loss_type == 'OSL':
        loss_fn = losses.OSLCELoss()
    
    if loss_type == 'OSL':
        Data.include_weak(Weak.w)
    else:
        Data.include_weak(Weak.z)

    trainloader,testloader = Data.get_dataloader(weak_labels = 'weak')




    #trainloader,testloader = Data.get_dataloader()
    mlp = MLP(Data.num_features, [Data.num_features], Weak.c, dropout_p=0.3, bn = True, activation='relu')
    optim = torch.optim.Adam(mlp.parameters(), lr=1e-3)
    mlp,results = train_and_evaluate(mlp,trainloader,testloader,optimizer=optim,loss_fn=loss_fn,corr_p=corr_p,num_epochs=100,sound=10,rep=i)







# Bwd = FwdBwdLoss(pinv(M),I_c)
# Fwd = FwdBwdLoss(I_d,M)

c:\Users\danibacaicoa\vscode_projects\ForwardBackard_losses\.venv\Lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.

Epoch 10/100: Train Loss: 3.4998, Train Acc: 0.2679, Test Acc: 0.2619, Train Detached Loss: 0.0635, Test Detached Loss: 0.0803, Learning Rate: 0.001000
Epoch 20/100: Train Loss: 3.4357, Train Acc: 0.2798, Test Acc: 0.2619, Train Detached Loss: 0.0605, Test Detached Loss: 0.0758, Learning Rate: 0.001000
Epoch 30/100: Train Loss: 3.3517, Train Acc: 0.3512, Test Acc: 0.3810, Train Detached Loss: 0.0572, Test Detached Loss: 0.0719, Learning Rate: 0.001000
Epoch 40/100: Train Loss: 3.2387, Train Acc: 0.3810, Test Acc: 0.3810, Train Detached Loss: 0.0520, Test Detached Loss: 0.0651, Learning Rate: 0.001000
Epoch 50/100: Train Loss: 3.1730, Train Acc: 0.4167, Test Acc: 0.4048, Train Detached Loss: 0.0479, Test Detached Loss: 0.0601, Learning Rate: 0.001000
Epoch 60/100: Train Loss: 3.0865, Train Acc: 0.4345, Test Acc: 0.4048, Train Detached Loss: 0.0443, Test Detached Loss: 0.0564, Learning Rate: 0.001000
Epoch 70/100: Train Loss: 3.0173, Train Acc: 0.5417, Test Acc: 0.5476, Train Detached Lo

In [5]:

base_dir = "Results"
if corr_n is not None:
    folder_path = os.path.join(base_dir, f'{dataset}_{corruption}_p_+{corr_p}p_-{corr_n}')
else:
    folder_path = os.path.join(base_dir, f'{dataset}_{corruption}_p{corr_p}')

os.makedirs(folder_path, exist_ok=True)

# Specify the file name
file_name = f'{loss_type}.csv'
file_path = os.path.join(folder_path, file_name)
results.to_csv(file_path, index=False)

print(f'DataFrame saved as CSV at: {file_path}')


DataFrame saved as CSV at: Results\image_pll_p0.5\Forward.csv


In [6]:
results

,epoch,train_loss,train_acc,test_acc,train_detached_loss,test_detached_loss,optimizer,loss_fn,repetition,initial_lr,actual_lr,corr_p
0,1,3.469290,0.196429,0.142857,0.066955,0.090892,Adam,FwdBwdLoss,9,0.001,0.001,0.5
1,2,3.441603,0.285714,0.214286,0.065961,0.089786,Adam,FwdBwdLoss,9,0.001,0.001,0.5
2,3,3.413559,0.285714,0.261905,0.065322,0.088971,Adam,FwdBwdLoss,9,0.001,0.001,0.5
3,4,3.388708,0.285714,0.285714,0.064642,0.088314,Adam,FwdBwdLoss,9,0.001,0.001,0.5
4,5,3.371164,0.285714,0.285714,0.063951,0.087665,Adam,FwdBwdLoss,9,0.001,0.001,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,2.707886,0.583333,0.500000,0.035038,0.062313,Adam,FwdBwdLoss,9,0.001,0.001,0.5
96,97,2.713032,0.607143,0.500000,0.034726,0.061899,Adam,FwdBwdLoss,9,0.001,0.001,0.5
97,98,2.714207,0.595238,0.547619,0.034487,0.061459,Adam,FwdBwdLoss,9,0.001,0.001,0.5
98,99,2.714451,0.601190,0.547619,0.034288,0.061044,Adam,FwdBwdLoss,9,0.001,0.001,0.5
